In [57]:
import pandas as pd
import numpy as np
from datetime import datetime
import csv
import influxdb_client, os, time
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate a Token from the "Tokens Tab" in the UI
token = "JxCGieLiMyN7lNeNnOiPQstwDdw3-diYqbDQvfbVz9lohjSVvGqFbvcqp2xN_s5TCgaYi4koju3TZhmZE-WrxQ=="
org = "lab"
url = "http://10.118.126.244:8086/"
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org) #set client
query_api = client.query_api()
query = """from(bucket: "ESP32")
|> range(start: -14d, stop:now())
|> filter(fn: (r) => r["_measurement"] == "Energy_status")
|> filter(fn: (r) => r["_field"] == "W")
|> timeShift(duration: 8h)
|> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
|> drop(columns: ["_start","_stop","_measurement","_field"])
|> yield(name: "mean")"""
data_frame = query_api.query_data_frame(query, org="lab")
df=data_frame.drop(["result","table"], axis=1)
print(df)

c:\Users\Nic\anaconda3\envs\python310\lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    from(bucket: "ESP32")
|> range(start: -14d, stop:now())
|> filter(fn: (r) => r["_measurement"] == "Energy_status")
|> filter(fn: (r) => r["_field"] == "W")
|> timeShift(duration: 8h)
|> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
|> drop(columns: ["_start","_stop","_measurement","_field"])
|> yield(name: "mean") |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/univ

                                 _time  _value
0            2022-11-16 00:37:00+00:00   37.61
1            2022-11-16 00:38:00+00:00   37.74
2            2022-11-16 00:39:00+00:00   46.95
3            2022-11-16 00:40:00+00:00   47.91
4            2022-11-16 00:41:00+00:00   47.01
...                                ...     ...
18657        2022-11-28 23:37:00+00:00   36.88
18658        2022-11-28 23:38:00+00:00   35.51
18659        2022-11-28 23:39:00+00:00   35.65
18660        2022-11-28 23:40:00+00:00   36.15
18661 2022-11-28 23:40:24.045217+00:00   35.91

[18662 rows x 2 columns]
